# Case Study 1 - Data Modality

```Check if I should use debug method instead of timegan```

## The Task
Get used to loading datasets with the library and generating synthetic data from them, whatever the modality of the real data.

### Imports
Lets get the imports out of the way. We import the required standard and 3rd party libraries and relevant Synthcity modules. We can also set the level of logging here, using Synthcity's bespoke logger. 

In [ ]:
# Standard
import sys
import warnings
from pathlib import Path

# 3rd party
import numpy as np
import pandas as pd

# synthcity
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import (GenericDataLoader, SurvivalAnalysisDataLoader, TimeSeriesDataLoader,TimeSeriesSurvivalDataLoader)

# Configure warnings and logging
warnings.filterwarnings("ignore")

# Set the level for the logging
# log.add(sink=sys.stderr, level="DEBUG")
log.remove()

# Loading data of different modalities

In this notebook we will load different datasets into synthcity and show that data of many different modalities can be used to generate synthetic data using this module.

### Static Data
Now we will start with the simplest example, static tabular data. For this, we will use the diabetes dataset from sklearn. First, we need to load the dataset.

In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y
display(X)

Then we pass it to the `GenericDataLoader` object from `synthcity`.

In [ ]:
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

We can print out different methods that are compatible with our data by calling `Plugins().list()` with a relevant list passed to the categories parameter.

In [ ]:
print(Plugins(categories=["generic"]).list())

No need to worry about the code in this next block here, we will go into lots of detail in how to generate synthetic data in the case studies to come. It is here purely to demonstrate that our dataset can be used to generate synthetic data using the synthcity module. We are using the method `marginal_distributions` to generate the synthetic data, which is one of the available debugging methods.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10).dataframe()

## Static survival
Next lets look at censored data. Censoring is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event. Censoring is common in survival analysis. For our next example we will load a static survival dataset. Our dataset this time is a veteran lung cancer dataset provided by scikit-survival. 

First, load the dataset.

In [ ]:
from sksurv.datasets import load_veterans_lung_cancer

data_x, data_y = load_veterans_lung_cancer()
data_x["status"], data_x["survival_in_days"] = [record[0] for record in data_y], [record[1] for record in data_y]
display(data_x)

Pass it to the DataLoader. This time we will use the `SurvivalAnalysisDataLoader`. We need to pass it the data, the name of the column that contains our labels or targets to `target_column` and the the name of the column  containing the time elapsed when the event occurred (the event defined by the target column) to `time_to_event_column`. Calling `info()` on the loader object allows us to see the information about the dataset we have just prepared.

In [ ]:

loader = SurvivalAnalysisDataLoader(
    data_x,
    target_column="status",
    time_to_event_column="survival_in_days",
)
print(loader.info())


If we get the `marginal_distributions` plugin again and fit it to the `loader` object, we can then call `generate` to produce the synthetic data.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10)

### Regular Time Series

In this next example we will load up a simple regular time series dataset and show that it is compatible with Synthcity. The temporal data must be passed to the loader as a list of dataframes, where each dataframe in the list refers to a different record and contains all time points for the record. So, there is a small amount of pre-processing to get our data into the right shape. As it is a regular time series we can simply pass a sequential list for each record.

The dataset we will use here is the basic motions dataset provided by SKTime. So, we need to import the library.

In [ ]:
from sktime.datasets import load_basic_motions

Load the data and re-format it into a list of dataframes, where each dataframe in the list refers to a different record and contains all time points for the record. We also need the outcomes as a dataframe and the observation times as a list of time steps for each record. As this is a regular time series our time steps can simply be a sequential list of integers.

In [ ]:

X, y = load_basic_motions(
    split="TRAIN", return_X_y=True, return_type="pd-multiindex"
)
num_instances = len(set((x[0] for x in X.index)))
num_time_steps = len(set((x[1] for x in X.index)))

temporal_data = [X.loc[i] for i in range(num_instances)]
y = pd.DataFrame(y, columns=["label"])
observation_times = [list(range(num_time_steps)) for i in range(num_instances)]

Pass the data we just prepared to the DataLoader. Here we will use the `TimeSeriesDataLoader`. Then we will print out the loader info to check everything looks correct.

In [ ]:
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    outcome=y,
)
display(loader.dataframe())
print(loader.info())

Now we are ready to produce the synthetic data. We will use the `timegan` plugin to handle the timeseries data. As we don't care about the quality of the dataset here, we just want to check that it is compatible and practice loading datasets, we can pass `n_iter=1` to limit the number of iterations in the generator.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)

### Irregular Time Series

Now lets load an irregular time series dataset and show that that is also compatible with Synthcity. The dataset we will use here is a google stocks dataset provided by the synthcity module itself.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader

static_data, temporal_data, observation_times, outcome = GoogleStocksDataloader().load()

As the dataset is wrapped by synthcity, it is already provided to us in the correct format, but the requirements are the same as before. The temporal data is a list of dataframes, where each dataframe in the list refers to a different record and contains all time points for the record. The outcomes are all in one dataframe and the observation times are a list of time steps for each record. The main difference here is that the observation times is a list of floats that represent the time between each data point.

In [ ]:
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=static_data,
    outcome=outcome,
)
print(loader.info())
display(loader.dataframe())

Exactly as for the regular time series, we can now generate synthetic data, by selecting our time series compatible plugin, then calling `fit()` and `generate()`.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=5)

### Composite Irregular Time Series Survival Analysis

In this final example we will look at composite data while adding all the other more complex elements we have looked at so far. This next dataset is a composite irregular time series survival analysis dataset. 

Again this dataset is provided by synthcity, so there is little to do in terms of pre-processing as everything is in the right format to begin with.

In [ ]:
from synthcity.utils.datasets.time_series.pbc import PBCDataloader
(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()

Even complex datasets such as this are compatible with Synthcity. We can load this data using the `TimeSeriesSurvivalDataLoader`. Then by calling `loader.info()`, we can check the information about the dataset. It contains both one static feature ("sex") and 14 temporal features, making it a composite dataset. The `seq_time_id` field shows the irregular time sampling, which we create by passing the values to the `observation_times` parameter of the `TimeSeriesSurvivalDataLoader` object. And finally, we are formulating this data as a survival analysis problem, which is indicated by the presence of a `time_to_event` field.

In [ ]:
T, E = outcome_surv

horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(T, horizons).tolist()

loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    observation_times=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
    time_horizons=time_horizons,
)

print(loader.info())

We can now generate synthetic data, in the way we are now well familiar with. We select our time series compatible plugin, then call `fit()` and `generate()`.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=5)

### Create synthetic datasets
 1) Above we have generated data with the debugging method `"marginal_distributions"` for tabular data and `timegan` for time series data. Now, using `Plugins().list()` or the documentation find another method that is compatible with some of the datasets to see if you can generate your own snthetic data.
 
 2) Generate synthetic data for another dataset of your choice using the methods described above. You can use any of the other dataset from the sources we have used above: [SKLearn](https://scikit-learn.org/stable/datasets/toy_dataset.html), [SKTime](https://www.sktime.org/en/stable/api_reference/datasets.html), [SKSurv](https://scikit-survival.readthedocs.io/en/stable/api/datasets.html)  or [synthcity](https://github.com/vanderschaarlab/synthcity/tree/main/src/synthcity/utils/datasets) itself.